In [71]:
!pip3 install nltk
!pip3 install stanfordcorenlp
!pip3 install harvesttext

# https://github.com/blmoistawinde/HarvestText

    100% |████████████████████████████████| 1.8MB 932kB/s 
    100% |████████████████████████████████| 778kB 2.1MB/s 
    100% |████████████████████████████████| 10.1MB 196kB/s 
    100% |████████████████████████████████| 61kB 8.3MB/s 
    100% |████████████████████████████████| 348kB 4.0MB/s 
    100% |████████████████████████████████| 512kB 3.7MB/s 
    100% |████████████████████████████████| 51kB 10.9MB/s 
    100% |████████████████████████████████| 71kB 10.3MB/s 
  Running setup.py bdist_wheel for pyxDamerauLevenshtein ... error
  Complete output from command /home/terry/pan/github/ai_writer/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-slzfr3wu/pyxDamerauLevenshtein/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmpg0b3yvtppip-wheel- --python-tag cp36:
  usage: -c [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: -c --help [cmd1 cmd

In [1]:
# 切换工作目录
import os
os.chdir("../")   #修改当前工作目录
os.getcwd()    #获取当前工作目录

'/home/terry/pan/github/ai_writer/ai_writer'

In [6]:
# # -*- coding:utf-8 -*-
# Author：wancong
# Date: 2018-04-29
from pyhanlp import *


def demo_dependency_parser():
    """ 依存句法分析（CRF句法模型需要-Xms512m -Xmx512m -Xmn256m，
        MaxEnt和神经网络句法模型需要-Xms1g -Xmx1g -Xmn512m）
        https://github.com/hankcs/pyhanlp
    """
    sentence = HanLP.parseDependency("天喂饲食物的量要适度，过多过少都不利成长。")
#     print(dir())
    for word in sentence.iterator():  # 通过dir()可以查看sentence的方法
#         print(word.ID)
#         print(word.HEAD.ID)
        print((word.ID,word.LEMMA, word.DEPREL, word.HEAD.LEMMA,word.POSTAG,word.HEAD.ID))
    print()

    # 也可以直接拿到数组，任意顺序或逆序遍历
    word_array = sentence.getWordArray()
    
    for word in word_array:
#         print("%s --(%s)--> %s" % (word.LEMMA, word.DEPREL, word.HEAD.LEMMA))
        print((word.LEMMA, word.DEPREL, word.HEAD.LEMMA,word.POSTAG,word.HEAD.ID))
    print(len(word_array))

    # 还可以直接遍历子树，从某棵子树的某个节点一路遍历到虚根
    CoNLLWord = JClass("com.hankcs.hanlp.corpus.dependency.CoNll.CoNLLWord")
    head = word_array[0]
    while head.HEAD:
        head = head.HEAD
        if (head == CoNLLWord.ROOT):
            print(head.LEMMA)
        else:
            print("%s --(%s)--> " % (head.LEMMA, head.DEPREL))

if __name__ == "__main__":
    demo_dependency_parser()
#     import doctest
#     doctest.testmod(verbose=True, optionflags=doctest.NORMALIZE_WHITESPACE)

java.util.NoSuchElementExceptionPyRaisable: java.util.NoSuchElementException

In [59]:
class WordBean(object):
    ''' 扩展conllword，存储父节点与孩子节点索引
    https://zekizz.github.io/NLP/information-extraction/
    '''
    def __init__(self):
       self.lemma = None
       self.postag = None
       self.relation = None
       self.head_id = None
       self.flag = True  # 是否还处于树中标志位，已合并的ATT将置为False
       self.child = []

    def set_word(self, conll_word):
       self.lemma = conll_word.LEMMA
       self.postag = conll_word.POSTAG
       self.relation = conll_word.DEPREL
       self.head_id = conll_word.HEAD.ID - 1

    def add_child(self, child_id, child_relation):
       self.child.append((child_id, child_relation))
       
   
    def extract_entity_tuple(self, dependency_tree, seed_id, head_type=''):
       '''自上而下解析树'''
       res_entity_tuple_list = []
       res_dict = dict()
       res_dict['time'] = []
       res_dict['reason'] = []
       res_dict['place'] = []
       res_dict['status'] = []
       # 判断是否是叶子节点
       if len(dependency_tree[seed_id].child) == 0:
           if dependency_tree[seed_id].lemma in self.status_set:
               res_dict['status'].append((seed_id, dependency_tree[seed_id].lemma))
           elif self.check_is_time(dependency_tree, seed_id):
               res_dict['time'].append((seed_id, dependency_tree[seed_id].lemma))
           elif head_type == 'reason':
               res_dict['reason'].append((seed_id, dependency_tree[seed_id].lemma))
           elif head_type == 'time':
               res_dict['time'].append((seed_id, dependency_tree[seed_id].lemma))
           else:
               if not dependency_tree[seed_id].lemma in self.discard_word_set:
                   res_dict['place'].append((seed_id, dependency_tree[seed_id].lemma))
           res_entity_tuple_list.append(res_dict)
           return res_entity_tuple_list

       # 非叶子节点需要向下递归解析
       if dependency_tree[seed_id].lemma in self.status_set:
           # 当前节点为状态节点
           status_merge_list = []
           for c_id, c_relation in dependency_tree[seed_id].child:
               child_bean = dependency_tree[c_id]
               if c_relation in ('COO', '并列关系'):
                   # 假设状态下不存在嵌套状态，有视为补充
                   if dependency_tree[c_id].postag == 'v' and len(dependency_tree[c_id].child) == 0:
                       res_dict['status'].append((c_id, dependency_tree[c_id].lemma))
                   else:
                       child_dict_list = self.extract_entity_tuple(dependency_tree, c_id)
                       for child_dict in child_dict_list:
                           self.merge_two_tuple_dict(res_dict, child_dict)
               elif c_relation in ('ADV', '状中结构'):
                   # 处理状中结构
                   if child_bean.lemma in ('因', '受', '由于'):
                       # 处理原因
                       child_dict_list = self.extract_entity_tuple(dependency_tree, c_id)
                       for child_dict in child_dict_list:
                           self.merge_two_tuple_dict(res_dict, child_dict)
                   elif child_bean.lemma == '处' or child_bean.postag == 'p':
                       child_dict_list = self.extract_entity_tuple(dependency_tree, c_id)
                       for child_dict in child_dict_list:
                           self.merge_two_tuple_dict(res_dict, child_dict)
                   elif child_bean.postag in ('a', 'ad', 'd'):
                       self.merge_att(dependency_tree, c_id)
                       status_merge_list.append(c_id)
                   elif self.check_is_time(dependency_tree, c_id):
                       self.merge_att(dependency_tree, c_id)
                       res_dict['time'].append((c_id, dependency_tree[c_id].lemma))
               elif c_relation in ('POB', '介宾关系') and child_bean.lemma in ('因', '受', '由于'):
                   self.merge_att(dependency_tree, c_id)
                   res_dict['reason'].append((c_id, dependency_tree[c_id].lemma))
               elif c_relation in ('CMP', '动补结构'):
                   self.merge_att(dependency_tree, c_id)
                   status_merge_list.append(c_id)
               elif c_relation in ('SBV', '主谓关系'):
                   # 处理主谓关系，解析具体地点
                   child_dict_list = self.extract_entity_tuple(dependency_tree, c_id)
                   for child_dict in child_dict_list:
                       self.merge_two_tuple_dict(res_dict, child_dict)
               elif c_relation in ('VOB', '动宾关系'):
                   if len(dependency_tree[c_id].child) == 0:
                       res_dict['status'].append((c_id, dependency_tree[c_id].lemma))
                   else:
                       child_dict_list = self.extract_entity_tuple(dependency_tree, c_id)
                       for child_dict in child_dict_list:
                           self.merge_two_tuple_dict(res_dict, child_dict)

           status_buffer = []
           status_merge_list.append(seed_id)
           status_merge_list.sort()
           for id in status_merge_list:
               status_buffer.append(dependency_tree[id].lemma)
           res_dict['status'].append((seed_id, ''.join(status_buffer)))

           res_entity_tuple_list.append(res_dict)
           return res_entity_tuple_list
       else:
           # 当前节点为非状态节点
           pre_head_type = head_type
           if self.check_is_time(dependency_tree, seed_id):
               # 为时间节点
               head_type = 'time'
               res_dict['time'].append((seed_id, dependency_tree[seed_id].lemma))
           elif dependency_tree[seed_id].lemma in ('因', '受', '由于') or head_type == 'reason':
               # 为原因节点
               head_type = 'reason'
               res_dict['reason'].append((seed_id, dependency_tree[seed_id].lemma))
           else:
               if not dependency_tree[seed_id].lemma in self.discard_word_set:
                   res_dict['place'].append((seed_id, dependency_tree[seed_id].lemma))
           child_dict_list = []
           coo_list = []
           for c_id, c_relation in dependency_tree[seed_id].child:
               # if c_relation in ('COO', '并列关系'):
               #     coo_list.append(c_id)
               if c_relation in ('WP', '标点符号'):
                   continue
               else:
                   if head_type == 'reason' and not pre_head_type == 'reason':
                       if c_relation in ('POB', '介宾关系'):
                           child_dict_list.extend(self.extract_entity_tuple(dependency_tree, c_id, head_type))
                           if dependency_tree[c_id].lemma in self.status_set:
                               res_dict['reason'].append((c_id, dependency_tree[c_id].lemma))
                   else:
                       child_dict_list.extend(self.extract_entity_tuple(dependency_tree, c_id, head_type))

           # 先合并非状态
           status_dict_list = []
           for child_dict in child_dict_list:
               if len(child_dict['status']) > 0:
                   status_dict_list.append(child_dict)
               else:
                   self.merge_two_tuple_dict(res_dict, child_dict)

           # 再合并存在状态的
           if len(status_dict_list) == 0:
               res_entity_tuple_list.append(res_dict)
           else:
               for child_dict in status_dict_list:
                   # tmp_dict = res_dict.copy()
                   tmp_dict = copy.deepcopy(res_dict)
                   self.merge_two_tuple_dict(tmp_dict, child_dict)
                   res_entity_tuple_list.append(tmp_dict)

           return res_entity_tuple_list


    def extract_information(self, line):
#        segs = self.nlp_tokenizer.seg(line)
#        # fix segs
#        self.fix_seged_postag(segs)
#        conll_words = self.parser.parse(segs).getWordArray()
       conll_words = line
       dependency_tree, root_id = self.construct_dependency_tree(conll_words)

       res_entity.append(self.extract_entity_tuple(dependency_tree, i))
       res_entity = self.extract_entity_tuple(dependency_tree, root_id)

       # print entity tuples
       for entity in res_entity:
           entity['time'].sort()
           entity['place'].sort()
           entity['reason'].sort()
           entity['status'].sort()
           self.fix_entity_tuple_dict(entity)
           print(entity)
       return res_entity




In [66]:
sentence = HanLP.parseDependency("天喂饲食物的量要适度，过多过少都不利成长。")
w = WordBean()

# print(w.extract_information(sentence))
sentence = HanLP.parseDependency("天喂饲食物的量要适度，过多过少都不利成长。")
#     print(dir())
# for word in sentence.iterator():  # 通过dir()可以查看sentence的方法
# #         print(word.ID)
# #         print(word.HEAD.ID)
# #     print((word.ID,word.LEMMA, word.DEPREL, word.HEAD.LEMMA,word.POSTAG,word.HEAD.ID))
#     print(w.extract_information(word))
# # print()
# word_array = sentence.getWordArray()

# print(w.extract_information(word_array))

word_array = sentence.getWordArray()

for word in word_array:
#         print("%s --(%s)--> %s" % (word.LEMMA, word.DEPREL, word.HEAD.LEMMA))
    print((word.LEMMA, word.DEPREL, word.HEAD.LEMMA,word.POSTAG,word.HEAD.ID))
# print(len(word_array))
#     print(w.extract_information(word_array))
    print(w.extract_entity_tuple(word_array, word.ID))
print(w.extract_entity_tuple(word_array))

('天', '定中关系', '食物', 'n', 3)


AttributeError: 'com.hankcs.hanlp.corpus.dependency.CoNll.CoNLLWord' object has no attribute 'child'

In [72]:
# from nltk.tree import Tree
# from stanfordcorenlp import StanfordCoreNLP
# sentence = '我叫小米'
# with StanfordCoreNLP(r'E:\ProgramData\Anaconda3\coreNLP\stanford-corenlp-full-2016-10-31', lang='zh') as nlp:
#     Tree.fromstring(nlp.parse(sentence)).draw()
# # --------------------- 
# # 作者：闰土不用叉 
# # 来源：CSDN 
# # 原文：https://blog.csdn.net/xyz1584172808/article/details/81951846 
# # 版权声明：本文为博主原创文章，转载请附上博文链接！

OSError: E:\ProgramData\Anaconda3\coreNLP\stanford-corenlp-full-2016-10-31 is not a directory.

In [3]:
from harvesttext import HarvestText
ht = HarvestText()

In [4]:
ht0 = HarvestText()
para = "上港的武磊武球王是中国最好的前锋。"
entity_mention_dict = {'武磊': ['武磊', '武球王'], "上海上港":["上港"]}
entity_type_dict = {'武磊': '球员', "上海上港":"球队"}
ht0.add_entities(entity_mention_dict, entity_type_dict)
for arc in ht0.dependency_parse(para):
    print(arc)
print(ht0.triple_extraction(para))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.692 seconds.
Prefix dict has been built succesfully.


java.util.NoSuchElementExceptionPyRaisable: java.util.NoSuchElementException